In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Assignment 1 Question 3

### a) Simulate the following function

$$ v=a_0+a_1t+a_2\sin{a_4t}+a_3\cos{a_4t}$$

Include Gaussian errors

In [2]:
def fit_function(t, *params):
    return params[0] + params[1]*t + params[2]*np.sin(params[4]*t) + params[3]*np.sin(params[4]*t)